# Analysis.

In [128]:
import pandas as pd
from pandas import json_normalize

import requests 
import json
import os
from dotenv import load_dotenv #conda install -c conda-forge python-dotenv

import time
import importlib
import src.extraction as ex
importlib.reload(ex)

<module 'src.extraction' from 'd:\\ironhack\\proyectos\\PipeLine_proy2\\src\\extraction.py'>

## 1. USDA food database.
### 1.1. Exploration.

Did a research of all the USDA csv files and select only the more relevant for this proyect, they have a lot of information about comples dishes, lab tests, market prices, etc. 1. Searching for the right tables with value information for the proyect. 2. Cleaning each table, droping the columns and probably some rows with no need it information. ----try to merge it in python by changing the name of tha columns to the same in each. exported as csv 3. Merge the tables -- prbably on SQL exported again in csv and put it back on python.

#### 1.1.1. "Food csv".
Data frame of food, it has its id number and name of food.

In [129]:
url = "D:\ironhack\proyectos\PipeLine_proy2\\USDA_main_csv\\food.csv"

In [130]:
food = ex.import_csv (url)
food.sample()

,fdc_id,data_type,description,food_category_id,publication_date
136849,424416,branded_food,"GIANT, CANDY CANES, CHERRY",NaN,2019-04-01


In [131]:
food.shape

(1185090, 5)

##### 1.1.1.1. Deleting.
Will remove the "food_category_id", "data_type" and the "publication_date" column and try to remove the index column, it appears that it does not do much, it doest have a column name for identification

In [132]:
food = ex.delete_food (food)
food

,fdc_id,description
0,1105904,WESSON Vegetable Oil 1 GAL
1,1105905,SWANSON BROTH BEEF
2,1105906,CAMPBELL'S SLOW KETTLE SOUP CLAM CHOWDER
3,1105907,CAMPBELL'S SLOW KETTLE SOUP CHEESE BROCCOLI
4,1105898,Discrepancy between the Atwater factor predict...
...,...,...
1185077,1757377,"MUSHROOMS, SHIITAKE"
1185078,1757378,"MUSHROOMS, SHIITAKE"
1185079,1757379,"MUSHROOMS, SHIITAKE"
1185080,1757380,"MUSHROOMS, SHIITAKE"


In [133]:
food.duplicated().sum()

0

#### 1.1.2. "Nutrients Table".
In this table can relate the data of nutrients and in wich food they are. they specifie the mount of the nutrient per 100g of food. Specified in unit defined in the nutrient table.

In [134]:
url2 = "D:\ironhack\proyectos\PipeLine_proy2\\USDA_main_csv\\nutrient.csv"
nutrient = ex.import_csv(url2)
nutrient.sample()

,id,name,unit_name,nutrient_nbr,rank
302,1306,TFA 18:2 t not further defined,G,665.0,15610.0


In [135]:
nutrient.shape

(463, 5)

##### 1.1.2.1. Deleting.
will delete must of the columns, except name and id, but will create a copy so i dont lose the information, could help me in the future.

In [136]:
nutrient = ex.delete_nutrient (nutrient)
nutrient.sample()

,id,name
206,1209,"Polyphenols, total"


In [137]:
nutrient.shape

(463, 2)

#### 1.1.3. "Food_id Nutrient_id"
This table doesn´t appear in the information docs, but it loocks really helfull allready matching the nutrient with the food and the portion per 100g. Wont delete any column, but will start to change names of each Data Frame to matched them later. 

In [138]:
url3 = "D:\ironhack\proyectos\PipeLine_proy2\\USDA_main_csv\\JCS_FoodId-NutrientId-Per100g.csv"
ides = ex.import_csv (url3)
ides.sample()

,FoodId,NutrientId,Per100g
9342109,1284840,203,0.0


### 1.2. Merging.
Will merge all data frames into one, will chose the right tables for it and clean it more.

Looking for same elements in different tables at the ID rows, will change the names to matched them.

#### 1.2.1. Rename.

In [139]:
# change decription to food
# change fdc_id to food_id
food.rename(columns= {"fdc_id" : "food_id", "description" : "food"}, inplace=True)
food.sample()

,food_id,food
1021610,1593909,GUMMY BEARS


In [140]:
#change NutrientId to nutrient_id
#change foodId to food_id
ides.rename(columns= {"FoodId" : "food_id", "NutrientId" : "nutrient_id", 'Per100g ' : "Per100g"}, inplace=True)
ides.sample()

,food_id,nutrient_id,Per100g
7074024,1095997,1008,367.0


In [141]:
# change id to nutrient_id
# change name to nutrient
nutrient.rename(columns= {"name" : "nutrient", "id" : "nutrient_id"}, inplace=True)
nutrient.sample()

,nutrient_id,nutrient
72,1074,Xylose


#### 1.2.2. Merge.
Will do a inner merge, now with the same column name. also cleaned some other columns by doing drop, the id columns.

In [142]:
usda = ex.merge (nutrient, ides, food)
usda.sample()

d:\ironhack\proyectos\PipeLine_proy2\src\extraction.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usda.drop(columns = ["food_id", "nutrient_id"], inplace= True, axis=1)


,food,nutrient,Per100g
6203663,"BUFFALO WING CRUNCHY BROAD BEANS, BUFFALO WING","Sugars, total including NLEA",7.14


In [143]:
usda.shape

(7555511, 3)

In [144]:
usda.duplicated().sum()

2243090

In [145]:
usda.drop_duplicates(keep='first')

,food,nutrient,Per100g
0,Hummus,Nitrogen,1.28
1,Hummus,Total lipid (fat),19.00
2,Hummus,Ash,1.98
3,Hummus,Water,56.30
4,Hummus,Nitrogen,1.29
...,...,...,...
7555478,"OIL, OLIVE, EXTRA LIGHT",Delta-7-Stigmastenol,21.90
7555479,"OIL, PEANUT",Stigmastadiene,0.00
7555487,"OIL, SUNFLOWER",Stigmastadiene,0.00
7555495,"OIL, SAFFLOWER",Stigmastadiene,0.00


In [146]:
usda["nutrient"] = [i.strip().replace("  ", " ") for i in list(usda["nutrient"])]
usda["food"] = [i.strip().replace("  ", " ") for i in list(usda["food"])]

In [147]:
usda.drop_duplicates(subset=['food', 'nutrient'], keep='first', inplace= True)

In [148]:
usda.shape

(4731512, 3)

In [149]:
usda.Per100g.value_counts()

0.000        1463803
3.570          31016
10.000         27565
1.000          25760
25.000         22938
              ...   
41.090             1
2411.000           1
20438.000          1
13.606             1
12.095             1
Name: Per100g, Length: 26134, dtype: int64

In [150]:
usda[["food", "Per100g", "nutrient"]].groupby(by=["food", "nutrient"]).agg({"nutrient":"value_counts"})

nutrient
food                                        nutrient                       nutrient                                
!AJUA!, CAFFEINE FREE SODA, MANDARIN ORANGE Carbohydrate, by difference    Carbohydrate, by difference            1
                                            Energy                         Energy                                 1
                                            Protein                        Protein                                1
                                            Sodium, Na                     Sodium, Na                             1
                                            Sugars, total including NLEA   Sugars, total including NLEA           1
...                                                                                                             ...
{TRAIL MIX}                                 Sodium, Na                     Sodium, Na                             1
                                            Sugars, total including NLEA   Sugars, total including NLEA           1
                                            Total lipid (fat)              Total lipid (fat)                      1
                                            Vitamin A, IU                  Vitamin A, IU                          1
                                            Vitamin C, total ascorbic acid Vitamin C, total ascorbic acid         1

[4731512 rows x 1 columns]

## 2. HEALTH CARE web scraping.
### 2.1. Exploration.
Got the information from the page vedantu, about some nutrients and what causes the lak of them.
#### 2.1.1. Get the link.

In [151]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import lxml
import importlib
import src.extraction as ex
importlib.reload(ex)

<module 'src.extraction' from 'd:\\ironhack\\proyectos\\PipeLine_proy2\\src\\extraction.py'>

In [152]:
url = "https://www.vedantu.com/biology/deficiency-diseases"
tags = ex.get_text(url)

Filter the elements to the info we need.

In [153]:
tag_1 = tags[37:55]
tag_2 = tags[60:68]
tag_3 = tags[73:91]

In [154]:
info_1 = ex.text_filtered(tag_1)
info_2 = ex.text_filtered(tag_2)
info_3 = ex.text_filtered(tag_3)

Creating the lists into a data frame, it made it into a single column frame, had to separate the information into 2 columns, nutrients and the information, and at the end put the 3 data frames into 1.

In [155]:
df = ex.db_concat (info_1, info_2, info_3)
df

,nutrients,info
0,thiamine (vitamin b1),severe deficiency causes beriberi with symptom...
1,riboflavin (vitamin b2),the deficiency of this vitamin causes angular ...
2,niacin (vitamin b3),causes pellagra which is a life-threatening di...
3,pantothenic acid (vitamin b5),vitamin b5 deficiency is rare and the symptoms...
4,vitamin b6,end-stage kidney diseases such as celiac disea...
5,biotin (vitamin b7),the deficiency affects the growth of hair and ...
6,folate (vitamin b9),it is very common and the deficiency is associ...
7,vitamin b12,deficiency of this disease leads to pernicious...
8,ascorbic acid (vitamin c),deficiency results in scurvy and the symptoms ...
0,retinol (vitamin a),the deficiency causes night blindness or nycta...


Had to do the searching for the disease manually.

In [156]:
lsttotal = list(df["info"])
lsttotal[20]

'the deficiency of phosphorus leads to the formation of underdeveloped teeth and bones.'

In [157]:
lsttotal = list(df["info"])
lst0 = lsttotal[0]
lst1 = lsttotal[1]
lst2 = lsttotal[2]
lst3 = lsttotal[3]
lst4 = lsttotal[4]
lst5 = lsttotal[5]
lst6 = lsttotal[6]
lst7 = lsttotal[7]
lst8 = lsttotal[8]
lst9 = lsttotal[9]
lst10 = lsttotal[10]
lst11 = lsttotal[11]
lst12 = lsttotal[12]
lst13 = lsttotal[13]
lst14 = lsttotal[14]
lst15 = lsttotal[15]
lst16 = lsttotal[16]
lst17 = lsttotal[17]
lst18 = lsttotal[18]
lst19 = lsttotal[19]
lst20 = lsttotal[20]
lst21 = lsttotal[21]


In [158]:
dis0 = ex.diseases (lst0)
dis1 = ex.diseases (lst1)
dis2 = ex.diseases (lst2)
dis3 = ex.diseases (lst3)
dis4 = ex.diseases (lst4)
dis5 = ex.diseases (lst5)
dis6 = ex.diseases (lst6)
dis7 = ex.diseases (lst7)
dis8 = ex.diseases (lst8)
dis9 = ex.diseases (lst9)
dis10 = ex.diseases (lst10)
dis11 = ex.diseases (lst11)
dis12 = ex.diseases (lst12)
dis13 = ex.diseases (lst13)
dis14 = ex.diseases (lst14)
dis15 = ex.diseases (lst15)
dis16 = ex.diseases (lst16)
dis17 = ex.diseases (lst17)
dis18 = ex.diseases (lst18)
dis19 = ex.diseases (lst19)
dis20 = ex.diseases (lst20)
dis21 = ex.diseases (lst21)

In [159]:
lista = ex.diseases_total (dis0, dis1, dis2, dis3, dis4, dis5, dis6, dis7, dis8, dis9, dis10, dis11, dis12, dis13, dis14, dis15, dis16, dis17, dis18, dis19, dis20, dis21) 

Makiing them a data frame.

In [162]:
enf = ex.df_diseases (df, lista)
enf.sample()

TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

Merging the frames.

In [ ]:
disess = ex.df_concat (df, enf)
disess

,nutrients,info,0
0,thiamine (vitamin b1),severe deficiency causes beriberi with symptom...,NaN
1,riboflavin (vitamin b2),the deficiency of this vitamin causes angular ...,NaN
2,niacin (vitamin b3),causes pellagra which is a life-threatening di...,NaN
3,pantothenic acid (vitamin b5),vitamin b5 deficiency is rare and the symptoms...,NaN
4,vitamin b6,end-stage kidney diseases such as celiac disea...,NaN
5,biotin (vitamin b7),the deficiency affects the growth of hair and ...,NaN
6,folate (vitamin b9),it is very common and the deficiency is associ...,NaN
7,vitamin b12,deficiency of this disease leads to pernicious...,NaN
8,ascorbic acid (vitamin c),deficiency results in scurvy and the symptoms ...,NaN
0,retinol (vitamin a),the deficiency causes night blindness or nycta...,NaN


In [ ]:
enf = ex.df_diseases (lista)